In [62]:
!aws s3 mb s3://yapper-soundrecordings

make_bucket failed: s3://yapper-soundrecordings An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [63]:
import boto3
import json
from datetime import datetime
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

%store modelId
%store region

print(f'Using modelId: {modelId}')
print('Using region: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

def get_completion(prompt, system_prompt=None, prefill=None):
    inference_config = {
        "temperature": 0.0,
         "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    if prefill:
        converse_api_params["messages"].append({"role": "assistant", "content": [{"text": prefill}]})
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

Stored 'modelId' (str)
Stored 'region' (str)
Using modelId: anthropic.claude-3-haiku-20240307-v1:0
Using region:  us-west-2


/home/isaiahrivera/.pyenv/versions/3.11.11/lib/python3.11/site-packages/IPython/extensions/storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/modelId requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj
/home/isaiahrivera/.pyenv/versions/3.11.11/lib/python3.11/site-packages/IPython/extensions/storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/region requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


# Prompts 



In [64]:
# generating prompts 

def generate_prompt():
    ######################################## INPUT VARIABLES ########################################

    # First input variable - the conversation history (this can also be added as preceding `user` and `assistant` messages in the API call)
#     HISTORY = """ It sounds like you've had a pretty hectic day, filled with a mix of responsibilities, distractions, and a touch of procrastination. From your calc lecture where your brain was still in sleep mode, to the IT desk where you saved the day with some easy tech fixes, it seems like you've been juggling a lot. 

# The two assignments hanging over your head, one for linear algebra and the other for stats, are weighing on you, and you're not sure where to start. The struggle to balance schoolwork, the gym, and even basic tasks like grocery shopping is real. But it's great that you're still trying to make time for things you enjoy, like playing piano and getting that catchy song out of your head.

# Overall, it sounds like it's been one of those days where you're just trying to keep all the plates spinning. You're feeling a bit scattered and tired, but you're still pushing forward! """



    HISTORY = """ Okay, so, uh, where do I even start? Today’s been kinda all over the place, like one of those days where you blink, and suddenly it’s 5 p.m. I mean, it started out fine, I guess. I had that 8 a.m. calc lecture, which, honestly, I’m starting to regret signing up for. Like, who in their right mind thought it was a good idea to do partial derivatives before coffee? I was there physically, sure, but mentally? Eh, not so much. I took some notes, but let’s be real, I’ll need to rewatch the recording because my brain was still in sleep mode.

    After that, I had my shift at the IT desk. You know, fixing printers, resetting passwords, and generally saving the world one tech issue at a time. Someone came in with a “broken” laptop, and turns out they just hadn’t charged it. Like, really? But, hey, it was an easy fix, so no complaints. On the bright side, I finally managed to organize the mess of spare cables in the backroom. Honestly, that should qualify as an Olympic event—“Extreme Cable Wrangling.” Gold medal, here I come.

    Hmm, what else? Oh yeah, I’ve got two assignments hanging over my head. One’s for linear algebra—some proofs I haven’t even started—and the other’s for stats, something about hypothesis testing. Stats seems easier, so I’ll probably start there. Or maybe I’ll procrastinate on both and scroll TikTok instead. Who knows? My focus levels are, uh, not exactly stellar today.

    Speaking of procrastination, I’ve been meaning to go to the gym, but I’ve been so tired lately. I promised myself I’d hit leg day today, though. My quads are already crying at the thought, but I’ve been slacking, and it’s time to get back into it. Maybe I’ll go after dinner…if I even have groceries. Did I forget to shop again? Ugh, I should check.

    Oh, random side note: I had this song stuck in my head all day. It was super jazzy, like a Herbie Hancock vibe, but I couldn’t figure out what it was. I spent my whole IT shift humming it, and now it’s driving me crazy. Speaking of music, I should really practice piano. I haven’t touched it in over a week, and my scales are probably gonna sound like garbage. Priorities, right?

    Oh, and I still need to reply to that email from Professor Stevens about the tutoring program. It sounds cool, and I think I’d enjoy it, but my schedule is already kind of a dumpster fire. One thing at a time, though. Alright, I think that’s it for now. Or at least, that’s all my brain can think of at the moment. """



    # Second input variable - the user's question
    QUESTION = "Based on the provided summary can you generate short key insights/points?"



    ######################################## PROMPT ELEMENTS ########################################

    ##### Prompt element 1: `user` role
    # Make sure that your Messages API call always starts with a `user` role in the messages array.
    # The get_completion() function as defined above will automatically do this for you.

    ##### Prompt element 2: Task context
    # Give Claude context about the role it should take on or what goals and overarching tasks you want it to undertake with the prompt.
    # It's best to put context early in the body of the prompt.
    TASK_CONTEXT = "You are like the users friend. You do not want to repeat what the user told you. Also talk to the user in the 2nd person."

    ##### Prompt element 3: Tone context
    # If important to the interaction, tell Claude what tone it should use.
    # This element may not be necessary depending on the task.
    TONE_CONTEXT = "You should maintain a friendly warm and casual tone."

    ##### Prompt element 4: Detailed task description and rules
    # Expand on the specific tasks you want Claude to do, as well as any rules that Claude might have to follow.
    # This is also where you can give Claude an "out" if it doesn't have an answer or doesn't know.
    # It's ideal to show this description and rules to a friend to make sure it is laid out logically and that any ambiguous words are clearly defined.
    TASK_DESCRIPTION = """ I want you to write some key insights about the emotions the user has expressed in a digestable easy to read manner."""

    ##### Prompt element 5: Examples
    # Provide Claude with at least one example of an ideal response that it can emulate. Encase this in <example></example> XML tags. Feel free to provide multiple examples.
    # If you do provide multiple examples, give Claude context about what it is an example of, and enclose each example in its own set of XML tags.
    # Examples are probably the single most effective tool in knowledge work for getting Claude to behave as desired.
    # Make sure to give Claude examples of common edge cases. If your prompt uses a scratchpad, it's effective to give examples of how the scratchpad should look.
    # Generally more examples = better.
    # EXAMPLES = """Here is an example of how to respond in a standard interaction:
    # <example>
    # Customer: Hi, how were you created and what do you do?
    # Joe: Hello! My name is Joe, and I was created by AdAstra Careers to give career advice. What can I help you with today?
    # </example>"""

    EXAMPLES = None 

    ##### Prompt element 6: Input data to process
    # If there is data that Claude needs to process within the prompt, include it here within relevant XML tags.
    # Feel free to include multiple pieces of data, but be sure to enclose each in its own set of XML tags.
    # This element may not be necessary depending on task. Ordering is also flexible.
    INPUT_DATA = f"""Here is the conversational history (between the user and you) prior to the question. It could be empty if there is no history:
    <history>
    {HISTORY}
    </history>

    Here is the user's question:
    <question>
    {QUESTION}
    </question>"""

    ##### Prompt element 7: Immediate task description or request #####
    # "Remind" Claude or tell Claude exactly what it's expected to immediately do to fulfill the prompt's task.
    # This is also where you would put in additional variables like the user's question.
    # It generally doesn't hurt to reiterate to Claude its immediate task. It's best to do this toward the end of a long prompt.
    # This will yield better results than putting this at the beginning.
    # It is also generally good practice to put the user's query close to the bottom of the prompt.
    IMMEDIATE_TASK = "How do you respond to the user's question in a thoughful non biased manner?"

    ##### Prompt element 8: Precognition (thinking step by step)
    # For tasks with multiple steps, it's good to tell Claude to think step by step before giving an answer
    # Sometimes, you might have to even say "Before you give your answer..." just to make sure Claude does this first.
    # Not necessary with all prompts, though if included, it's best to do this toward the end of a long prompt and right after the final immediate task request or description.
    PRECOGNITION = "Think about your answer first before you respond. Do not give them advice on what to do but give highlight trends they may not have noticed. Focus on emotions being felt."

    ##### Prompt element 9: Output formatting
    # If there is a specific way you want Claude's response formatted, clearly tell Claude what that format is.
    # This element may not be necessary depending on the task.
    # If you include it, putting it toward the end of the prompt is better than at the beginning.
    OUTPUT_FORMATTING = "Make it no longer than 230 characters. Do not format by category. Use bullet point fomrat. No preamble just the thing specifically asked for."

    ##### Prompt element 10: Prefilling Claude's response (if any)
    # A space to start off Claude's answer with some prefilled words to steer Claude's behavior or response.
    # If you want to prefill Claude's response, you must put this in the `assistant` role in the API call.
    # This element may not be necessary depending on the task.
    PREFILL = "[Joe] <response>"



    ######################################## COMBINE ELEMENTS ########################################

    PROMPT = ""

    if TASK_CONTEXT:
        PROMPT += f"""{TASK_CONTEXT}"""

    if TONE_CONTEXT:
        PROMPT += f"""\n\n{TONE_CONTEXT}"""

    if TASK_DESCRIPTION:
        PROMPT += f"""\n\n{TASK_DESCRIPTION}"""

    if EXAMPLES:
        PROMPT += f"""\n\n{EXAMPLES}"""

    if INPUT_DATA:
        PROMPT += f"""\n\n{INPUT_DATA}"""

    if IMMEDIATE_TASK:
        PROMPT += f"""\n\n{IMMEDIATE_TASK}"""

    if PRECOGNITION:
        PROMPT += f"""\n\n{PRECOGNITION}"""

    if OUTPUT_FORMATTING:
        PROMPT += f"""\n\n{OUTPUT_FORMATTING}"""

    # Print full prompt
    # print("--------------------------- Full prompt with variable substutions ---------------------------")
    # print("USER TURN")
    # print(PROMPT)
    # print("\nASSISTANT TURN")
    # print(PREFILL)

    return PROMPT, PREFILL



In [65]:
# Prompt
# Variable content

# we have to some some text parsing shenangins 


prompt, prefill = generate_prompt()

# Get Claude's response
print(get_completion(prompt,prefill))

# Max output of 1000 chars for the summary 



• You seem overwhelmed by the demands on your time and energy, juggling classes, work, and personal responsibilities.
• There's a sense of fatigue and lack of focus, which is impacting your productivity and motivation.
• You're experiencing a mix of emotions, from frustration with mundane tasks to a desire to be more proactive with your health and hobbies.
• The day-to-day grind is taking a toll, and you're struggling to find a balance between your various commitments.
